<a href="https://colab.research.google.com/github/9M3a1h3d9i9/Machine_Learning_Practice_Master/blob/Develope/Proj_2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In The name of GOD

# Step 1: Import Libraries and Load Dataset

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns


# Load your dataset:


In [10]:
df = pd.read_csv('sales_data_sample.csv', encoding='latin1')
# UTF-8 Error
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


# Step 2: Data Preprocessing

# 1.   Handle Missing Values




Impute missing values using SimpleImputer from Scikit-learn (mean/median for numerical data, mode for categorical).

In [6]:
from sklearn.impute import SimpleImputer
num_imputer = SimpleImputer(strategy='mean')
cat_imputer = SimpleImputer(strategy='most_frequent')



2.   Split Data into Train/Test


In [12]:
X = data.drop(columns=['SALES'])
y = data['SALES']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# 3.  Feature Scaling



In [13]:
scaler = StandardScaler()


# 4. Convert Categorical to Numerical


In [14]:
ohe = OneHotEncoder(drop='first')


# 5. Combine Preprocessing Steps

In [19]:
numerical_features = ['ORDERLINENUMBER', 'QUANTITYORDERED', 'PRICEEACH', 'ORDERDATE', 'QTR_ID', 'MONTH_ID', 'YEAR_ID', 'PRODUCTCODE',  'MSRP', 'DEALSIZE']
categorical_features = ['COUNTRY', 'TERRITORY', 'STATUS', 'PRODUCTLINE', 'CUSTOMERNAME']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', scaler, numerical_features),
        ('cat', ohe, categorical_features)
    ]
)

# Build a Custom Transformer

In [20]:
from sklearn.base import BaseEstimator, TransformerMixin

class CustomFeatureTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X['ad_to_prod_ratio'] = X['advertising'] / X['production_quantity']  # Example feature
        return X


# Create a Pipeline

In [21]:
# pipeline = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('feature_engineering', CustomFeatureTransformer()),
#     ('model', LinearRegression())
# ])
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('feature_engineering', CustomFeatureTransformer()), # Assuming this transformer is defined correctly and its features are in your data
    ('model', LinearRegression())
])


# Perform Cross-Validation

In [23]:
from sklearn.model_selection import cross_val_score

cv = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring='neg_mean_squared_error')
print("Cross-validated MSE:", -cv_scores.mean())


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 469, in fit
    Xt = self._fit(X, y, routed_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 406, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/usr/local/lib/python3.10/dist-packages/joblib/memory.py", line 312

# Step 7: Hyperparameter Tuning
# Use GridSearchCV and RandomizedSearchCV to tune parameters for ridge and lasso:

In [ ]:
param_grid = {
    'model': [Ridge(), Lasso()],
    'model__alpha': [0.1, 1, 10, 100]  # Example hyperparameters
}
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

print("Best Params:", grid_search.best_params_)
print("Best MSE:", -grid_search.best_score_)


# Model Evaluation
Evaluate performance on the test set:

In [ ]:
from sklearn.metrics import mean_squared_error

y_pred = grid_search.best_estimator_.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"Test MSE: {mse:.2f}, Test RMSE: {rmse:.2f}")


# Answers to Questions
1. Why is feature scaling important?
Feature scaling ensures that all numerical features contribute equally to the model by bringing them to a similar range, especially for distance-based algorithms or when gradient descent optimization is used.

2. What preprocessing method is recommended for missing values?
For linear regression, numerical features can be imputed using the mean/median. For categorical features, the mode (most frequent) is typically used.

3. Difference between simple partitioning and KFold cross-validation?
Simple partitioning evaluates the model once, which might not reflect its generalizability. KFold splits data into multiple folds, evaluates on each, and provides a more robust estimate.

4. Parameter vs. Hyperparameter?
Parameter: Model-determined values (e.g., weights in linear regression).
Hyperparameter: Manually set values (e.g., alpha in Ridge regression).
5. GridSearchCV vs. RandomizedSearchCV?
GridSearchCV: Tests all possible parameter combinations (more exhaustive but slower).
RandomizedSearchCV: Randomly tests a subset (faster, good for larger search spaces).
6. Best method for optimizing linear regression and feature selection?
Use Ridge/Lasso Regression for regularization and feature selection. These methods penalize unnecessary features and prevent overfitting.

